[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Digital-AI-Finance/Digital-Finance-Introduction/blob/main/day_03/notebooks/NB05_Cryptographic_Operations.ipynb)

# NB05: Cryptographic Operations

**Topic:** 3.1 - Cryptographic Building Blocks

**Learning Objectives:**
- Understand hash functions and their properties
- Generate public-private key pairs
- Create and verify digital signatures
- Connect cryptographic primitives to blockchain applications

## 1. Setup

We'll use Python's built-in `hashlib` for hashing and the `cryptography` library for public-key operations.

In [ ]:
!pip install -q cryptography

In [ ]:
import hashlib
import os
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.backends import default_backend

print("✓ Libraries loaded successfully")

## 2. Hash Functions

### What is a Hash Function?

A **cryptographic hash function** takes an input (of any size) and produces a fixed-size output called a **hash** or **digest**.

**Key Properties:**
1. **Deterministic**: Same input always produces same output
2. **One-way**: Cannot reverse the hash to get the original input
3. **Fixed output size**: SHA-256 always produces 256-bit (32-byte) output
4. **Avalanche effect**: Tiny input change completely changes the output
5. **Collision resistant**: Extremely hard to find two inputs with same hash

### SHA-256 Demonstration

In [ ]:
# Hash a simple message
message = "Hello, Digital Finance!"
message_bytes = message.encode('utf-8')

# Create SHA-256 hash
hash_object = hashlib.sha256(message_bytes)
hash_hex = hash_object.hexdigest()

print(f"Original message: {message}")
print(f"SHA-256 hash:     {hash_hex}")
print(f"Hash length:      {len(hash_hex)} characters (64 hex = 256 bits)")

### The Avalanche Effect

Let's see what happens when we change just ONE character:

In [ ]:
# Original message
message1 = "Hello, Digital Finance!"
hash1 = hashlib.sha256(message1.encode()).hexdigest()

# Change one character: ! -> ?
message2 = "Hello, Digital Finance?"
hash2 = hashlib.sha256(message2.encode()).hexdigest()

print("Message 1:", message1)
print("Hash 1:   ", hash1)
print()
print("Message 2:", message2)
print("Hash 2:   ", hash2)
print()

# Count how many characters differ
differences = sum(c1 != c2 for c1, c2 in zip(hash1, hash2))
print(f"Difference: {differences} out of 64 characters changed ({differences/64*100:.1f}%)")

**Observation:** Changing just one character changes about 50% of the hash! This makes it impossible to predict how input changes affect the output.

### Exercise: Hash Your Own Messages

Try hashing different messages and see the results:

In [ ]:
def hash_message(msg):
    """Hash a message and display the result"""
    hash_result = hashlib.sha256(msg.encode()).hexdigest()
    print(f"Message: {msg}")
    print(f"Hash:    {hash_result}")
    print()
    return hash_result

# Try different messages
hash_message("Bitcoin")
hash_message("bitcoin")  # lowercase
hash_message("")         # empty string

## 3. Public Key Cryptography

### Asymmetric Encryption

**Public-key cryptography** uses a pair of mathematically related keys:
- **Private key**: Kept secret, used to sign and decrypt
- **Public key**: Shared openly, used to verify signatures and encrypt

**Key Idea:** What one key encrypts, only the other can decrypt.

In blockchain:
- Your **private key** = your password (never share!)
- Your **public key** → derives your wallet address
- You **sign** transactions with private key
- Others **verify** your signature with public key

### Generate RSA Key Pair

In [ ]:
# Generate a 2048-bit RSA private key
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)

# Derive the public key from private key
public_key = private_key.public_key()

print("✓ RSA key pair generated")
print(f"  Private key size: 2048 bits")
print(f"  Public key derived from private key")

### View Key Structure

In [ ]:
# Serialize private key to PEM format
private_pem = private_key.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption()
)

# Serialize public key to PEM format
public_pem = public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)

print("PRIVATE KEY (keep secret!):")
print(private_pem.decode()[:200], "...\n")

print("PUBLIC KEY (safe to share):")
print(public_pem.decode())

### Exercise: Generate Your Own Key Pair

In [ ]:
# Your turn! Generate a new key pair
my_private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)

my_public_key = my_private_key.public_key()

# Get the key numbers
private_numbers = my_private_key.private_numbers()
public_numbers = my_public_key.public_numbers()

print("Your key pair generated!")
print(f"Public exponent (e):  {public_numbers.e}")
print(f"Modulus size (n):     {public_numbers.n.bit_length()} bits")

## 4. Digital Signatures

### What is a Digital Signature?

A **digital signature** proves:
1. **Authentication**: The message was signed by the holder of the private key
2. **Integrity**: The message hasn't been tampered with
3. **Non-repudiation**: The signer can't deny signing it

**Process:**
1. Hash the message
2. Encrypt the hash with your private key → signature
3. Anyone with your public key can verify the signature

### Sign a Message

In [ ]:
# Message to sign
message = b"I authorize transfer of 10 BTC to address XYZ"

# Sign the message with private key
signature = private_key.sign(
    message,
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)

print("Message signed successfully!")
print(f"Message:   {message.decode()}")
print(f"Signature: {signature.hex()[:80]}...")
print(f"Signature length: {len(signature)} bytes")

### Verify the Signature

In [ ]:
# Verify signature with public key
try:
    public_key.verify(
        signature,
        message,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    print("✓ Signature is VALID")
    print("  The message was signed by the holder of the private key")
    print("  The message has not been tampered with")
except Exception as e:
    print("✗ Signature is INVALID")
    print(f"  Error: {e}")

### What Happens with Wrong Key?

In [ ]:
# Generate a different key pair (attacker's keys)
wrong_private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)
wrong_public_key = wrong_private_key.public_key()

# Try to verify with wrong public key
try:
    wrong_public_key.verify(
        signature,
        message,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    print("✓ Signature valid")
except Exception:
    print("✗ Verification FAILED with wrong public key")
    print("  This proves the signature was not created by this key pair")

### What Happens with Tampered Message?

In [ ]:
# Attacker tries to modify the message
tampered_message = b"I authorize transfer of 100 BTC to address XYZ"  # Changed 10 -> 100

# Try to verify tampered message with original signature
try:
    public_key.verify(
        signature,
        tampered_message,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    print("✓ Signature valid")
except Exception:
    print("✗ Verification FAILED with tampered message")
    print("  Original: 10 BTC")
    print("  Tampered: 100 BTC")
    print("  The signature detects any modification!")

## 5. Real-World Application

### How Bitcoin/Ethereum Use These Primitives

**Blockchain transactions combine all three cryptographic tools:**

1. **Hash Functions (SHA-256, Keccak-256)**
   - Link blocks together in the chain
   - Create transaction IDs
   - Merkle trees for efficient verification
   - Proof-of-work mining

2. **Public-Key Cryptography (ECDSA)**
   - Each wallet has a private/public key pair
   - Private key controls the funds
   - Public key derives the wallet address

3. **Digital Signatures (ECDSA signatures)**
   - Every transaction must be signed with private key
   - Network verifies signature before accepting transaction
   - Prevents unauthorized spending

### Wallet Address Derivation (Simplified)

Let's simulate how an Ethereum-style address is derived from a public key:

In [ ]:
# Simplified address derivation (actual process uses ECDSA, not RSA)
def derive_address_simplified(public_key_bytes):
    """Simulate address derivation from public key"""
    # Hash the public key
    hash_result = hashlib.sha256(public_key_bytes).digest()
    
    # Take last 20 bytes (Ethereum uses Keccak256, we use SHA256 for demo)
    address_bytes = hash_result[-20:]
    
    # Convert to hex and add 0x prefix
    address = "0x" + address_bytes.hex()
    
    return address

# Derive address from our public key
address = derive_address_simplified(public_pem)

print("Wallet Address Derivation:")
print(f"  Public Key → Hash → Address")
print(f"  Address: {address}")
print()
print("Key Point: Anyone can see your address and public key,")
print("but only YOU know the private key that controls the funds.")

### Transaction Signing Flow

In [ ]:
# Simulate a blockchain transaction
transaction = {
    "from": address,
    "to": "0x742d35Cc6634C0532925a3b844Bc9e7595f0bEb",
    "amount": 1.5,
    "nonce": 42,
    "timestamp": "2024-01-15T10:30:00Z"
}

print("Transaction to sign:")
for key, value in transaction.items():
    print(f"  {key}: {value}")

# Serialize transaction for signing
tx_string = str(transaction)
tx_bytes = tx_string.encode()

# Sign the transaction
tx_signature = private_key.sign(
    tx_bytes,
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)

print()
print(f"✓ Transaction signed")
print(f"  Signature: {tx_signature.hex()[:60]}...")
print()
print("This signed transaction can now be broadcast to the network.")
print("Miners will verify the signature before including it in a block.")

## 6. Challenge Exercises

### Exercise 1: Hash Collision Probability

SHA-256 produces 2^256 possible hashes. Let's understand how secure this is:

In [ ]:
# Calculate the number of possible SHA-256 hashes
possible_hashes = 2**256

print("SHA-256 Security:")
print(f"  Possible hashes: {possible_hashes:.2e}")
print(f"  That's approximately {possible_hashes / 10**77:.0f} × 10^77")
print()
print("Comparison:")
print(f"  Atoms in observable universe: ~10^80")
print(f"  Grains of sand on Earth: ~10^24")
print()
print("Finding a collision by brute force would take longer than")
print("the age of the universe, even with all computers on Earth.")

### Exercise 2: Sign and Verify Your Own Message

Create your own message, sign it, and verify it:

In [ ]:
# TODO: Write your own message here
your_message = b"Your message here"

# TODO: Generate a new key pair
your_private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)
your_public_key = your_private_key.public_key()

# TODO: Sign the message
your_signature = your_private_key.sign(
    your_message,
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)

# TODO: Verify the signature
try:
    your_public_key.verify(
        your_signature,
        your_message,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    print("✓ Your signature verified successfully!")
except Exception as e:
    print(f"✗ Verification failed: {e}")

### Exercise 3: Explore Hash Properties

Test the properties of hash functions:

In [ ]:
# Property 1: Deterministic
msg = "blockchain"
hash_1 = hashlib.sha256(msg.encode()).hexdigest()
hash_2 = hashlib.sha256(msg.encode()).hexdigest()
print(f"Deterministic: {hash_1 == hash_2} (same input → same output)")
print()

# Property 2: Fixed size
short_msg = "a"
long_msg = "a" * 1000
hash_short = hashlib.sha256(short_msg.encode()).hexdigest()
hash_long = hashlib.sha256(long_msg.encode()).hexdigest()
print(f"Fixed size: {len(hash_short)} chars = {len(hash_long)} chars")
print()

# Property 3: Efficiency
import time
huge_msg = "x" * 1_000_000  # 1 MB
start = time.time()
hashlib.sha256(huge_msg.encode()).hexdigest()
elapsed = time.time() - start
print(f"Efficiency: Hashed 1 MB in {elapsed*1000:.2f} milliseconds")

## Summary

In this notebook, you learned:

1. **Hash Functions**: One-way, deterministic functions that create unique fingerprints
   - SHA-256 is the backbone of Bitcoin
   - Tiny input changes → completely different hash
   - Collision resistant and efficient

2. **Public-Key Cryptography**: Asymmetric encryption with key pairs
   - Private key = secret, controls funds
   - Public key = shareable, verifies signatures
   - Mathematically linked but cannot derive private from public

3. **Digital Signatures**: Proof of authenticity and integrity
   - Sign with private key
   - Verify with public key
   - Essential for blockchain transactions

4. **Real-World Usage**:
   - Every blockchain transaction uses all three primitives
   - Your wallet = private key
   - Your address = derived from public key
   - Your transactions = signed with private key

**Next Steps**: In the next notebooks, we'll see how these cryptographic building blocks enable Bitcoin, Ethereum, and other blockchain systems to function securely without central authorities.